In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mlsp-padded/reshaped/x_train548.npy
/kaggle/input/mlsp-padded/reshaped/x_train224.npy
/kaggle/input/mlsp-padded/reshaped/x_train768.npy
/kaggle/input/mlsp-padded/reshaped/x_train607.npy
/kaggle/input/mlsp-padded/reshaped/x_train1.npy
/kaggle/input/mlsp-padded/reshaped/x_train60.npy
/kaggle/input/mlsp-padded/reshaped/x_train945.npy
/kaggle/input/mlsp-padded/reshaped/x_train158.npy
/kaggle/input/mlsp-padded/reshaped/x_train706.npy
/kaggle/input/mlsp-padded/reshaped/x_train428.npy
/kaggle/input/mlsp-padded/reshaped/x_train229.npy
/kaggle/input/mlsp-padded/reshaped/x_train305.npy
/kaggle/input/mlsp-padded/reshaped/x_train578.npy
/kaggle/input/mlsp-padded/reshaped/x_train990.npy
/kaggle/input/mlsp-padded/reshaped/x_train950.npy
/kaggle/input/mlsp-padded/reshaped/x_train366.npy
/kaggle/input/mlsp-padded/reshaped/x_train605.npy
/kaggle/input/mlsp-padded/reshaped/x_train854.npy
/kaggle/input/mlsp-padded/reshaped/x_train620.npy
/kaggle/input/mlsp-padded/reshaped/x_train290.npy
/ka

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.utils import shuffle
import json

def get_data():
    lst_x = []
    lst_y = list(np.load("../input/mlsp-padded/reshaped/y_train.npy"))
    with open("../input/mlsp-padded/reshaped/Labels_dict") as f:
        dict1 = json.loads(f.read()) 
    for m in range(1000):
        lst_x.append(np.load("../input/mlsp-padded/reshaped/x_train" + str(m) + ".npy"))
    return lst_x, lst_y, dict1

def train_val_split(lst1, lst_y): # input shapes are: (1000, 128, 2584) and (1000, 10)
    count = 1
    x_train = []
    x_test = []
    y_train = []
    y_test = []
    for m in range(len(lst1)):
        if m <= (count*100) - 21:
            x_train.append(lst1[m])
            y_train.append(lst_y[m])
        else:
            x_test.append(lst1[m])
            y_test.append(lst_y[m])
        if m > (count*100) - 2:
            count = count + 1
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_test = np.array(x_test)
    y_test =  np.array(y_test)
    del(lst1)
    x_train, y_train = shuffle(x_train, y_train)
    x_test, y_test = shuffle(x_test, y_test)
    x_train = x_train[..., np.newaxis]
    x_test = x_test[..., np.newaxis]
    return x_train, x_test, y_train, y_test

In [4]:
x_train, y_train, dict1 = get_data()

In [5]:
x_train, x_test, y_train, y_test = train_val_split(x_train, y_train)

In [6]:
print(x_test.shape)
print(y_train.shape)
print(dict1)

(200, 128, 2584, 1)
(800, 10)
{'Bark': 0, 'Meow': 1, 'Siren': 2, 'Shatter': 3, 'Knock': 4, 'Crying_and_sobbing': 5, 'Microwave_oven': 6, 'Vehicle_horn_and_car_horn_and_honking': 7, 'Doorbell': 8, 'Walk_and_footsteps': 9}


In [7]:
(temp, f, t, num) = x_train.shape
print(f)
print(t)
print(num)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

128
2584
1
(800, 128, 2584, 1)
(200, 128, 2584, 1)
(800, 10)
(200, 10)


In [8]:
N_classes = 10
def make_model(f, t, num):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(f, t, num)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(2048, activation='leaky_relu'))
    model.add(tf.keras.layers.Dense(256, activation='leaky_relu'))
    #model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(64, activation='leaky_relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(N_classes, activation='softmax'))
    return model
model = make_model(f, t, num)

2022-10-12 12:30:56.810332: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-12 12:30:56.886972: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-12 12:30:56.887766: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-12 12:30:56.889608: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 330752)            0         
_________________________________________________________________
dense (Dense)                (None, 2048)              677382144 
_________________________________________________________________
dense_1 (Dense)              (None, 256)               524544    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                16448     
_________________________________________________________________
batch_normalization (BatchNo (None, 64)                256       
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                6

In [10]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.Precision(),tf.keras.metrics.Recall()],
)

# Train model for 50 epochs, capture the history
history = model.fit(x_train, y_train, verbose = 1, epochs=50, validation_data=(x_test, y_test))

2022-10-12 12:31:29.446506: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1058406400 exceeds 10% of free system memory.
2022-10-12 12:31:30.636700: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1058406400 exceeds 10% of free system memory.
2022-10-12 12:31:31.492822: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50
25/25 [==============================] - 5s 135ms/step - loss: 2.7432 - precision: 0.2395 - recall: 0.0500 - val_loss: 4.2005 - val_precision: 0.2091 - val_recall: 0.1150
Epoch 2/50
25/25 [==============================] - 2s 93ms/step - loss: 2.4046 - precision: 0.4029 - recall: 0.0700 - val_loss: 2.3446 - val_precision: 0.4375 - val_recall: 0.1400
Epoch 3/50
25/25 [==============================] - 2s 93ms/step - loss: 2.3643 - precision: 0.4143 - recall: 0.0725 - val_loss: 2.5552 - val_precision: 0.4194 - val_recall: 0.1300
Epoch 4/50
25/25 [==============================] - 2s 94ms/step - loss: 2.2788 - precision: 0.4643 - recall: 0.0812 - val_loss: 2.0091 - val_precision: 0.5227 - val_recall: 0.1150
Epoch 5/50
25/25 [==============================] - 2s 100ms/step - loss: 2.1779 - precision: 0.4867 - recall: 0.0913 - val_loss: 2.0413 - val_precision: 0.5000 - val_recall: 0.0700
Epoch 6/50
25/25 [==============================] - 2s 96ms/step - loss: 2.1008 - precision: 

In [ ]:
model.save('model_2_1.h5')

2022-10-12 01:04:06.898791: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2709520384 exceeds 10% of free system memory.


In [12]:
yhat_probs = model.predict(x_test, verbose=1)
y_hat=np.argmax(yhat_probs,axis=1)
print(y_hat)
y_target = []
for m in y_test:
    r = np.where(m == 1)
    y_target.append(int(r[0]))

7/7 [==============================] - 0s 29ms/step
[2 8 6 4 7 8 5 2 4 3 7 6 9 3 2 6 4 5 4 1 4 0 3 9 8 9 8 1 7 8 1 1 4 8 0 6 1
 1 9 1 9 7 2 1 1 9 0 1 3 1 9 0 5 6 6 1 5 3 8 0 8 2 4 1 1 0 4 0 1 0 8 1 8 5
 4 8 0 2 5 0 9 5 7 7 0 6 7 4 5 6 9 1 1 6 1 2 8 3 7 4 2 0 7 4 5 9 4 1 6 9 6
 7 1 9 3 5 1 8 2 7 8 5 8 2 6 8 1 9 9 5 1 1 8 5 7 6 7 5 6 0 2 1 9 7 0 5 2 7
 7 2 3 8 5 3 5 0 9 8 4 7 8 2 8 1 9 2 1 3 2 5 9 2 9 5 0 4 6 3 1 8 6 2 2 1 4
 1 1 0 8 7 2 7 8 9 0 4 7 3 8 8]


In [13]:
print(len(y_hat))
print(len(y_target))

200
200


In [14]:
from sklearn.metrics import classification_report
report = classification_report(y_target, y_hat, output_dict=True)
report_df = pd.DataFrame(report).transpose()

In [15]:
lst = []
for m in range(10):
    for key, value in dict1.items():
        if value == m:
            #print(key)
            lst.append(key)
lst.append("accuracy")
lst.append("macro avg")
lst.append("weighted avg")
report_df["Labels"] = lst

In [16]:
report_df

,precision,recall,f1-score,support,Labels
0,0.888889,0.80,0.842105,20.00,Bark
1,0.562500,0.90,0.692308,20.00,Meow
2,0.850000,0.85,0.850000,20.00,Siren
3,1.000000,0.60,0.750000,20.00,Shatter
4,0.764706,0.65,0.702703,20.00,Knock
5,0.684211,0.65,0.666667,20.00,Crying_and_sobbing
6,0.875000,0.70,0.777778,20.00,Microwave_oven
7,0.850000,0.85,0.850000,20.00,Vehicle_horn_and_car_horn_and_honking
8,0.730769,0.95,0.826087,20.00,Doorbell
9,0.550000,0.55,0.550000,20.00,Walk_and_footsteps


In [17]:
report_df.to_csv("model_2_result.csv")